In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torchvision.transforms as transforms
import numpy as np
import os
import time
import datetime

import sys
sys.path.append('..')
from Model_cyc import Generator_cyc, Discriminator_cyc
from Perceptual_loss import LossNetwork

from edge_promoting import edge_promoting

from utils import data_load
from utils import print_network
from utils import to_img
from utils import show_im
from utils import gram_matrix

In [2]:
# initialize
device = torch.device('cuda:1')

Data_dir = "/home/rongliang/CE7454_project/data/data_new/"
Output_dir = "/home/rongliang/CE7454_project/basic_181108/results_181114_same_GD_dim2_content10/"


img_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# network
G = Generator_cyc(domain_dim=2)
G = G.to(device)

D = Discriminator_cyc(domain_dim=2)
D = D.to(device)

with torch.no_grad():
    loss_network = LossNetwork()
    loss_network.to(device)
loss_network.eval()

# parameters
BATCH_SIZE = 8
NUM_EPOCH = 100
NUM_EPOCH_DECAY = 20
NUM_CRITIC = 5 # training D for n_critic times then training G once

IMG_SIZE = 128

LAMBDA_D_PROB = 1.
LAMBDA_D_GP = 10.

LAMBDA_G_CONTENT = 1e-4
LAMBDA_G_STYLE = 1e-6
LAMBDA_G_SMOOTH = 1e-6
LAMBDA_G_IDENTITY = 1.
LAMBDA_G_SR = 10.
LAMBDA_G_CYCLE = 1e-5

STYLE_WEIGHTS = np.array([1e-1, 1, 1e1, 5, 1e1]) * 1e4

G_lr_init = 0.0001
D_lr_init = 0.0001

# optimizer
g_optimizer = torch.optim.Adam(G.parameters(), lr=G_lr_init, betas=[0.5, 0.999])
d_optimizer = torch.optim.Adam(D.parameters(), lr=D_lr_init, betas=[0.5, 0.999])

# loss
criterion_L1 = torch.nn.L1Loss().to(device)
criterion_mse= torch.nn.MSELoss().to(device)



def update_lr(g_lr, d_lr, g_optimizer, d_optimizer):
    """Decay learning rates of the generator and discriminator."""
    for param_group in g_optimizer.param_groups:
        param_group['lr'] = g_lr
    for param_group in d_optimizer.param_groups:
        param_group['lr'] = d_lr
        
def gradient_penalty(y, x, device):
    """Compute gradient penalty: (L2_norm(dy/dx) - 1)**2."""
    weight = torch.ones(y.size()).to(device)
    dydx = torch.autograd.grad(outputs=y,
                               inputs=x,
                               grad_outputs=weight,
                               retain_graph=True,
                               create_graph=True,
                               only_inputs=True)[0]

    dydx = dydx.view(dydx.size(0), -1)
    dydx_l2norm = torch.sqrt(torch.sum(dydx**2, dim=1))
    return torch.mean((dydx_l2norm-1)**2)

# total variation
def compute_loss_smooth(mat):
    return torch.sum(torch.abs(mat[:, :, :, :-1] - mat[:, :, :, 1:])) + \
           torch.sum(torch.abs(mat[:, :, :-1, :] - mat[:, :, 1:, :]))

In [3]:
start = time.time()

Test_loader_src = data_load(os.path.join(Data_dir, 'src_data'), 'test', img_trans, BATCH_SIZE, shuffle=False, drop_last=True)
Test_iter = iter(Test_loader_src)
# train

for epoch in range(NUM_EPOCH):
    Train_loader_src = data_load(os.path.join(Data_dir, 'src_data'), 'train', img_trans, BATCH_SIZE, shuffle=True, drop_last=True)
    Train_loader_tgt = data_load(os.path.join(Data_dir, 'tgt_data'), 'pair', img_trans, BATCH_SIZE, shuffle=True, drop_last=True)
    
    ########## train
    i = -1
    for (x, _), (y, _) in zip(Train_loader_src, Train_loader_tgt):
        i = i + 1
        
#         e = y[:, :, :, IMG_SIZE:]
        y = y[:, :, :, :IMG_SIZE]
        
        ones_map = torch.ones([y.size(0),1])
        zeros_map = torch.zeros([y.size(0),1])
        
        x = x.to(device)
        y = y.to(device)
#         e = e.to(device)
        
        real_labels = torch.cat([ones_map, zeros_map], dim=1)
        cartoon_labels = torch.cat([zeros_map, ones_map], dim=1)
        
        real_labels = real_labels.to(device)
        cartoon_labels = cartoon_labels.to(device)
        
        ##### train D
        # real cartoon image
        D_patch_real_cartoon_D = D(y, cartoon_labels)
        
        # real human face images
        D_patch_real_human_D = D(x, real_labels)
        
#         # blur image
#         D_patch_blur_D = D(e)
        
        # fake image
        Fake_image_D = G(x, real_labels)
        D_patch_fake_cartoon_D = D(Fake_image_D.detach(), cartoon_labels)
        
        # cycle image
        Cycle_image_D = G(Fake_image_D, cartoon_labels)
        D_patch_fake_human_D = D(Cycle_image_D.detach(), real_labels)
           
        # compute loss
        D_loss_real_cartoon_D = -torch.mean(D_patch_real_cartoon_D) * LAMBDA_D_PROB
        D_loss_real_human_D = -torch.mean(D_patch_real_human_D) * LAMBDA_D_PROB
        
        D_loss_fake_cartoon_D = torch.mean(D_patch_fake_cartoon_D) * LAMBDA_D_PROB
        D_loss_fake_human_D = torch.mean(D_patch_fake_human_D) * LAMBDA_D_PROB
#         D_loss_blur_D = torch.mean(D_patch_blur_D) * LAMBDA_D_PROB
        
        # compute loss for gradient penalty
        alpha = torch.rand(y.size(0), 1, 1, 1).to(device)
        x_hat = (alpha * y.data + (1 - alpha) * Fake_image_D.data).requires_grad_(True)
        x_hat = x_hat.to(device)
        out_src = D(x_hat, cartoon_labels)
        D_loss_gp_cartoon_D = gradient_penalty(out_src, x_hat, device) * LAMBDA_D_GP
        
        #
        alpha = torch.rand(y.size(0), 1, 1, 1).to(device)
        x_hat = (alpha * x.data + (1 - alpha) * Cycle_image_D.data).requires_grad_(True)
        x_hat = x_hat.to(device)
        out_src = D(x_hat, real_labels)
        D_loss_gp_human_D = gradient_penalty(out_src, x_hat, device) * LAMBDA_D_GP
        
        # combine loss
        D_loss = D_loss_real_cartoon_D + D_loss_real_human_D \
                 + D_loss_fake_cartoon_D + D_loss_fake_human_D \
                 + D_loss_gp_cartoon_D + D_loss_gp_human_D 
    
        # update D
        d_optimizer.zero_grad()
        D_loss.backward()
        d_optimizer.step()
        
        ##### train G
        if (i + 1) % NUM_CRITIC == 0:
            # generate fake images
            Fake_image_G = G(x, real_labels)
            
            # cycle images
            Cycle_image_G = G(Fake_image_G, cartoon_labels)
            
            # loss with respect to D
            D_patch_fake_cartoon_G = D(Fake_image_G, cartoon_labels)
            D_loss_fake_cartoon_G = -torch.mean(D_patch_fake_cartoon_G) * LAMBDA_D_PROB
            
            D_patch_fake_human_G = D(Cycle_image_G, real_labels)
            D_loss_fake_human_G = -torch.mean(D_patch_fake_human_G) * LAMBDA_D_PROB
            
            # perceptual loss
            Input_img_features = loss_network(x.detach())
            Generated_img_features = loss_network(Fake_image_G)

#             Target_img_features = loss_network(y.detach())
            
            # content loss
            with torch.no_grad():
                Input_img_features_c = Input_img_features[2].detach()
                            
            G_loss_content_G = criterion_mse(Input_img_features.conv4_4.detach(), Generated_img_features.conv4_4) * LAMBDA_G_CONTENT
            
#             # style loss
#             style_loss = 0.
#             for l, weight in enumerate(STYLE_WEIGHTS):
#                 gram_s = gram_matrix(Target_img_features[l])
#                 gram_y = gram_matrix(Generated_img_features[l])
#                 style_loss += float(weight) * criterion_mse(gram_y, gram_s.expand_as(gram_y))
            
#             G_loss_style_G = style_loss * LAMBDA_G_STYLE
            
            # smoothness loss
            G_loss_Smoothness_G = compute_loss_smooth(Fake_image_G) * LAMBDA_G_SMOOTH
            
            # cycle loss
            G_loss_cycle_G = criterion_L1(x, Cycle_image_G) * LAMBDA_G_CYCLE
             
#             # identity loss
#             Identity_image_G = G(y)
#             G_loss_Identity_G = criterion_mse(y, Identity_image_G) * LAMBDA_G_IDENTITY
            
#             # super resolution loss
#             SR_image_G = G(e)
#             SR_loss_G = criterion_mse(e, SR_image_G) * LAMBDA_G_SR

            G_loss_style_G = torch.zeros(1).to(device)
#             G_loss_content_G = torch.zeros(1).to(device)
        
            G_loss = D_loss_fake_cartoon_G + D_loss_fake_human_G \
                     + G_loss_style_G + G_loss_content_G + G_loss_Smoothness_G \
                     + G_loss_cycle_G
            
            g_optimizer.zero_grad()
            G_loss.backward()
            g_optimizer.step()
            
        # print output
        if (i + 1) % 500 == 0:
            elapsed_total = time.time() - start  
            elapsed_total = str(datetime.timedelta(seconds=elapsed_total))[:-7]

            print('total elapsed time: {}, Epoch [{}/{}], Iter {}, D_loss_all: {:.6f}, G_loss_all: {:.6f}\n'
            'Train D:\nD_loss_real_cartoon_D: {:.6f}, D_loss_real_human_D: {:.6f}\n'
                  'D_loss_fake_cartoon_D: {:.6f}, D_loss_fake_human_D: {:.6f}\n'
                  'D_loss_gp_cartoon_D: {:.6f}, D_loss_gp_human_D: {:.6f}\n'
            'Train G:\nD_loss_fake_cartoon_G: {:.6f}, D_loss_fake_human_G: {:.6f}\n' 
                      'G_loss_style_G: {:.6f}, G_loss_content_G: {:.6f}\n'
                      'G_loss_Smoothness_G: {:.6f}, G_loss_cycle_G: {:.6f}'.format(
                  elapsed_total, epoch, NUM_EPOCH, i + 1, D_loss.item(), G_loss.item(), 
                  D_loss_real_cartoon_D.item(), D_loss_real_human_D.item(),
                  D_loss_fake_cartoon_D.item(), D_loss_fake_human_D.item(),
                  D_loss_gp_cartoon_D.item(), D_loss_gp_human_D.item(),
                  D_loss_fake_cartoon_G.item(), D_loss_fake_human_G.item(), 
                  G_loss_style_G.item(), G_loss_content_G.item(), 
                  G_loss_Smoothness_G.item(), G_loss_cycle_G.item()
                  ))        # attention image
            print(' ')

            # save images
            with torch.no_grad():
                Img_list = [x.detach()]
                Img_list.append(Fake_image_G.detach())
                Img_list.append(Cycle_image_G.detach())
              
                Img_concat = torch.cat(Img_list, dim = 2)
                Img_concat = to_img(Img_concat.cpu())

                save_image(Img_concat, Output_dir + 'images_epo{}_iter{}.png'.format(epoch+1, i+1), nrow=8, padding=0)

                
       
    # finish an epoch
    if (epoch + 1) > (NUM_EPOCH - NUM_EPOCH_DECAY):
        g_lr = G_lr_init - G_lr_init * (1/NUM_EPOCH_DECAY) * (epoch + 1 - (NUM_EPOCH - NUM_EPOCH_DECAY))
        d_lr = D_lr_init - D_lr_init * (1/NUM_EPOCH_DECAY) * (epoch + 1 - (NUM_EPOCH - NUM_EPOCH_DECAY))
        update_lr(g_lr, d_lr, g_optimizer, d_optimizer)
        
    # save images after a batch
    with torch.no_grad():
        Fake_image_B = G(x, real_labels)
        Cycle_image_B = G(Fake_image_B, cartoon_labels)
        
        Img_list = [x.detach()]
        Img_list.append(Fake_image_B.detach())
        Img_list.append(Cycle_image_B.detach())

        Img_concat = torch.cat(Img_list, dim = 2)
        Img_concat = to_img(Img_concat.cpu())

        save_image(Img_concat, Output_dir + 'images_epo{}_iter{}_batch.png'.format(epoch+1, i+1), nrow=8, padding=0)
    
    # save test images
    with torch.no_grad():
        Test_iter = iter(Test_loader_src) # fix testing samples
        TT = Test_iter.next()
        x = TT[0]
        x = x.to(device)
        
        Fake_image_T = G(x, real_labels)
        Cycle_image_T = G(Fake_image_T, cartoon_labels)
        Img_list = [x.detach()]
        Img_list.append(Fake_image_T.detach())
        Img_list.append(Cycle_image_T.detach())

        Img_concat = torch.cat(Img_list, dim = 2)
        Img_concat = to_img(Img_concat.cpu())

        save_image(Img_concat, Output_dir + 'images_epo{}_iter{}_test.png'.format(epoch+1, i+1), nrow=8, padding=0)
    
# save the model        
torch.save(G, Output_dir + 'G.pkl')
torch.save(D, Output_dir + 'D.pkl')            
        
            
            
        

total elapsed time: 0:06:17, Epoch [0/100], Iter 500, D_loss_all: -0.090582, G_loss_all: 9.088098
Train D:
D_loss_real_cartoon_D: -0.338191, D_loss_real_human_D: 0.029210
D_loss_fake_cartoon_D: -0.140135, D_loss_fake_human_D: -0.051078
D_loss_gp_cartoon_D: 0.213182, D_loss_gp_human_D: 0.196430
Train G:
D_loss_fake_cartoon_G: 0.180657, D_loss_fake_human_G: 0.076313
G_loss_style_G: 0.000000, G_loss_content_G: 7.601119
G_loss_Smoothness_G: 0.194668, G_loss_cycle_G: 1.035340
 
total elapsed time: 0:12:34, Epoch [0/100], Iter 1000, D_loss_all: 0.098464, G_loss_all: 9.606481
Train D:
D_loss_real_cartoon_D: -0.153632, D_loss_real_human_D: 0.370181
D_loss_fake_cartoon_D: -0.556769, D_loss_fake_human_D: -0.011392
D_loss_gp_cartoon_D: 0.123709, D_loss_gp_human_D: 0.326367
Train G:
D_loss_fake_cartoon_G: 0.571866, D_loss_fake_human_G: 0.045425
G_loss_style_G: 0.000000, G_loss_content_G: 7.834364
G_loss_Smoothness_G: 0.198884, G_loss_cycle_G: 0.955941
 
total elapsed time: 0:18:51, Epoch [0/100], 

total elapsed time: 1:59:21, Epoch [3/100], Iter 2000, D_loss_all: -8.586228, G_loss_all: 16.431505
Train D:
D_loss_real_cartoon_D: -0.576479, D_loss_real_human_D: 4.810299
D_loss_fake_cartoon_D: -8.248322, D_loss_fake_human_D: -6.526681
D_loss_gp_cartoon_D: 1.854311, D_loss_gp_human_D: 0.100645
Train G:
D_loss_fake_cartoon_G: 6.137341, D_loss_fake_human_G: 6.366102
G_loss_style_G: 0.000000, G_loss_content_G: 3.024146
G_loss_Smoothness_G: 0.077827, G_loss_cycle_G: 0.826091
 
total elapsed time: 2:05:38, Epoch [3/100], Iter 2500, D_loss_all: -12.616068, G_loss_all: 15.101177
Train D:
D_loss_real_cartoon_D: -5.277760, D_loss_real_human_D: 1.158782
D_loss_fake_cartoon_D: -4.785569, D_loss_fake_human_D: -5.314822
D_loss_gp_cartoon_D: 1.313186, D_loss_gp_human_D: 0.290115
Train G:
D_loss_fake_cartoon_G: 4.334964, D_loss_fake_human_G: 5.911424
G_loss_style_G: 0.000000, G_loss_content_G: 3.718285
G_loss_Smoothness_G: 0.086262, G_loss_cycle_G: 1.050244
 
total elapsed time: 2:11:55, Epoch [4/1

total elapsed time: 3:46:05, Epoch [7/100], Iter 500, D_loss_all: -15.061759, G_loss_all: 5.916921
Train D:
D_loss_real_cartoon_D: -13.092241, D_loss_real_human_D: -8.963041
D_loss_fake_cartoon_D: -3.934400, D_loss_fake_human_D: 6.435617
D_loss_gp_cartoon_D: 4.442795, D_loss_gp_human_D: 0.049512
Train G:
D_loss_fake_cartoon_G: 5.761693, D_loss_fake_human_G: -3.776948
G_loss_style_G: 0.000000, G_loss_content_G: 3.193865
G_loss_Smoothness_G: 0.074638, G_loss_cycle_G: 0.663673
 
total elapsed time: 3:52:22, Epoch [7/100], Iter 1000, D_loss_all: -12.397867, G_loss_all: 2.490665
Train D:
D_loss_real_cartoon_D: -4.999485, D_loss_real_human_D: -9.233705
D_loss_fake_cartoon_D: -5.658219, D_loss_fake_human_D: 3.516744
D_loss_gp_cartoon_D: 2.667570, D_loss_gp_human_D: 1.309229
Train G:
D_loss_fake_cartoon_G: 3.989505, D_loss_fake_human_G: -6.571280
G_loss_style_G: 0.000000, G_loss_content_G: 4.196272
G_loss_Smoothness_G: 0.066513, G_loss_cycle_G: 0.809655
 
total elapsed time: 3:58:39, Epoch [7/

total elapsed time: 5:39:07, Epoch [10/100], Iter 2000, D_loss_all: -13.629343, G_loss_all: 2.350770
Train D:
D_loss_real_cartoon_D: -12.906559, D_loss_real_human_D: -9.085997
D_loss_fake_cartoon_D: -4.400977, D_loss_fake_human_D: 7.029807
D_loss_gp_cartoon_D: 5.440741, D_loss_gp_human_D: 0.293640
Train G:
D_loss_fake_cartoon_G: 5.089808, D_loss_fake_human_G: -6.518803
G_loss_style_G: 0.000000, G_loss_content_G: 3.019131
G_loss_Smoothness_G: 0.073201, G_loss_cycle_G: 0.687432
 
total elapsed time: 5:45:24, Epoch [10/100], Iter 2500, D_loss_all: -5.218721, G_loss_all: 5.327667
Train D:
D_loss_real_cartoon_D: -1.919494, D_loss_real_human_D: -6.682656
D_loss_fake_cartoon_D: -4.149831, D_loss_fake_human_D: 6.397768
D_loss_gp_cartoon_D: 1.070353, D_loss_gp_human_D: 0.065140
Train G:
D_loss_fake_cartoon_G: 4.704128, D_loss_fake_human_G: -5.298308
G_loss_style_G: 0.000000, G_loss_content_G: 5.037280
G_loss_Smoothness_G: 0.073137, G_loss_cycle_G: 0.811431
 
total elapsed time: 5:51:41, Epoch [

total elapsed time: 7:25:53, Epoch [14/100], Iter 500, D_loss_all: -8.722360, G_loss_all: 3.520811
Train D:
D_loss_real_cartoon_D: -5.236911, D_loss_real_human_D: -10.207147
D_loss_fake_cartoon_D: -6.025797, D_loss_fake_human_D: 8.593753
D_loss_gp_cartoon_D: 3.955158, D_loss_gp_human_D: 0.198584
Train G:
D_loss_fake_cartoon_G: 7.371433, D_loss_fake_human_G: -7.571599
G_loss_style_G: 0.000000, G_loss_content_G: 2.963710
G_loss_Smoothness_G: 0.071757, G_loss_cycle_G: 0.685510
 
total elapsed time: 7:32:10, Epoch [14/100], Iter 1000, D_loss_all: -12.284706, G_loss_all: 0.594274
Train D:
D_loss_real_cartoon_D: -3.377775, D_loss_real_human_D: -15.221106
D_loss_fake_cartoon_D: -7.258115, D_loss_fake_human_D: 12.034039
D_loss_gp_cartoon_D: 1.409898, D_loss_gp_human_D: 0.128353
Train G:
D_loss_fake_cartoon_G: 7.341801, D_loss_fake_human_G: -12.542054
G_loss_style_G: 0.000000, G_loss_content_G: 5.055107
G_loss_Smoothness_G: 0.071696, G_loss_cycle_G: 0.667724
 
total elapsed time: 7:38:27, Epoch

...

total elapsed time: 12:46:10, Epoch [24/100], Iter 1000, D_loss_all: -7.180435, G_loss_all: 5.350496
Train D:
D_loss_real_cartoon_D: -3.435577, D_loss_real_human_D: -8.854705
D_loss_fake_cartoon_D: -8.021936, D_loss_fake_human_D: 8.195338
D_loss_gp_cartoon_D: 4.874443, D_loss_gp_human_D: 0.062002
Train G:
D_loss_fake_cartoon_G: 8.775489, D_loss_fake_human_G: -7.603477
G_loss_style_G: 0.000000, G_loss_content_G: 3.432499
G_loss_Smoothness_G: 0.072702, G_loss_cycle_G: 0.673283
 
total elapsed time: 12:52:27, Epoch [24/100], Iter 1500, D_loss_all: -10.669708, G_loss_all: 16.950415
Train D:
D_loss_real_cartoon_D: 2.435601, D_loss_real_human_D: -0.403370
D_loss_fake_cartoon_D: -12.545898, D_loss_fake_human_D: -0.831454
D_loss_gp_cartoon_D: 0.590394, D_loss_gp_human_D: 0.085019
Train G:
D_loss_fake_cartoon_G: 12.671707, D_loss_fake_human_G: 0.543506
G_loss_style_G: 0.000000, G_loss_content_G: 2.898339
G_loss_Smoothness_G: 0.067641, G_loss_cycle_G: 0.769222
 
total elapsed time: 12:58:43, Epo

total elapsed time: 14:33:00, Epoch [27/100], Iter 2000, D_loss_all: -9.971766, G_loss_all: 16.266865
Train D:
D_loss_real_cartoon_D: 0.907937, D_loss_real_human_D: -0.119808
D_loss_fake_cartoon_D: -10.827331, D_loss_fake_human_D: -1.157735
D_loss_gp_cartoon_D: 1.184872, D_loss_gp_human_D: 0.040299
Train G:
D_loss_fake_cartoon_G: 11.162135, D_loss_fake_human_G: 1.409309
G_loss_style_G: 0.000000, G_loss_content_G: 2.995513
G_loss_Smoothness_G: 0.068658, G_loss_cycle_G: 0.631250
 
total elapsed time: 14:39:18, Epoch [27/100], Iter 2500, D_loss_all: -6.540669, G_loss_all: 10.670082
Train D:
D_loss_real_cartoon_D: 2.463372, D_loss_real_human_D: -6.275225
D_loss_fake_cartoon_D: -9.038331, D_loss_fake_human_D: 5.210473
D_loss_gp_cartoon_D: 1.029970, D_loss_gp_human_D: 0.069072
Train G:
D_loss_fake_cartoon_G: 10.696136, D_loss_fake_human_G: -4.917105
G_loss_style_G: 0.000000, G_loss_content_G: 4.140534
G_loss_Smoothness_G: 0.078955, G_loss_cycle_G: 0.671562
 
total elapsed time: 14:45:36, Epo

total elapsed time: 16:19:51, Epoch [31/100], Iter 500, D_loss_all: -8.702003, G_loss_all: 4.246003
Train D:
D_loss_real_cartoon_D: -2.328387, D_loss_real_human_D: -8.176497
D_loss_fake_cartoon_D: -6.785571, D_loss_fake_human_D: 7.283336
D_loss_gp_cartoon_D: 1.285060, D_loss_gp_human_D: 0.020057
Train G:
D_loss_fake_cartoon_G: 7.006786, D_loss_fake_human_G: -7.223001
G_loss_style_G: 0.000000, G_loss_content_G: 3.800329
G_loss_Smoothness_G: 0.069274, G_loss_cycle_G: 0.592615
 
total elapsed time: 16:26:09, Epoch [31/100], Iter 1000, D_loss_all: -6.390334, G_loss_all: 6.694848
Train D:
D_loss_real_cartoon_D: 2.706943, D_loss_real_human_D: -6.865185
D_loss_fake_cartoon_D: -9.575769, D_loss_fake_human_D: 7.193241
D_loss_gp_cartoon_D: 0.097315, D_loss_gp_human_D: 0.053122
Train G:
D_loss_fake_cartoon_G: 10.255853, D_loss_fake_human_G: -6.794357
G_loss_style_G: 0.000000, G_loss_content_G: 2.452207
G_loss_Smoothness_G: 0.070608, G_loss_cycle_G: 0.710537
 
total elapsed time: 16:32:26, Epoch [

total elapsed time: 18:06:43, Epoch [34/100], Iter 1500, D_loss_all: -10.118114, G_loss_all: 12.530111
Train D:
D_loss_real_cartoon_D: -3.046848, D_loss_real_human_D: -3.197009
D_loss_fake_cartoon_D: -8.970234, D_loss_fake_human_D: 2.257860
D_loss_gp_cartoon_D: 2.814066, D_loss_gp_human_D: 0.024051
Train G:
D_loss_fake_cartoon_G: 9.548815, D_loss_fake_human_G: -1.655206
G_loss_style_G: 0.000000, G_loss_content_G: 3.842818
G_loss_Smoothness_G: 0.071399, G_loss_cycle_G: 0.722286
 
total elapsed time: 18:13:01, Epoch [34/100], Iter 2000, D_loss_all: -9.892656, G_loss_all: 10.793666
Train D:
D_loss_real_cartoon_D: -0.628275, D_loss_real_human_D: -5.323711
D_loss_fake_cartoon_D: -9.062707, D_loss_fake_human_D: 3.703731
D_loss_gp_cartoon_D: 1.377054, D_loss_gp_human_D: 0.041252
Train G:
D_loss_fake_cartoon_G: 9.990835, D_loss_fake_human_G: -3.387531
G_loss_style_G: 0.000000, G_loss_content_G: 3.536960
G_loss_Smoothness_G: 0.081968, G_loss_cycle_G: 0.571434
 
total elapsed time: 18:19:18, Epo

KeyboardInterrupt: 